In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = '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'
os.environ["HTTP_PROXY"] = '192.168.208.104:7890'
os.environ["HTTPS_PROXY"] = '192.168.208.104:7890'

hf = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en", model_kwargs={"device": "cpu"})
os.environ['HTTP_PROXY'] = ''
os.environ['HTTPS_PROXY'] = ''

/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If

In [2]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# 定义查询语句，查找所有节点及其名称
query = """
MATCH (n:Query)
RETURN n.name AS node_name
"""

# 执行查询并获取结果
def get_all_node_names(tx):
    result = tx.run(query)

    node_names = []
    for record in result:
        node_names.append(record['node_name'])
    
    return node_names

with driver.session() as session:
    all_node_names = session.read_transaction(get_all_node_names)
    print("All Node Names:", all_node_names)

# 关闭驱动
driver.close()

All Node Names: ['No Failure Under Precondition: A monad `m` does not fail under precondition `P`, if for no start state in that precondition it sets the failure flag.', 'seL4', 'Empty Fail: A well-formed monad returns an empty set of results only if it has the failure flag set.', 'Validity: A Hoare triple `{P} f {Q}` is valid if for all states that satisfy the precondition `P`, all result values and result states that are returned by the monad `f` satisfy the postcondition `Q`.', 'Non-failure and Validity: A monad `f` does not fail under precondition `P` and satisfies the Hoare triple `{P} f {Q}`.', 'Monad Operations: The monad operations `return`, `get`, and `put` behave as follows: `return x` does not change the state and returns `x`, `get` returns the current state, and `put s` changes the current state to `s`.', 'Bind Operation: The bind operation `f >>= g` is the execution of `f` followed by the execution of `g`, where `g` takes the result value and state of `f` as parameters.', 

/tmp/ipykernel_4129930/3191389499.py:29: DeprecationWarning: read_transaction has been renamed to execute_read
  all_node_names = session.read_transaction(get_all_node_names)


In [3]:
from langchain_community.vectorstores import Chroma
chroma = Chroma.from_texts(texts=all_node_names,embedding=hf)

In [4]:
retriever = chroma.as_retriever(search_kwargs={"k": 1})

In [5]:
def get_tree(tx, root_node_name):
    query = """
    MATCH (n {name: $root_node_name})
    OPTIONAL MATCH (n)-[:SPECIFY]->(child)
    RETURN n.name AS name, collect(child.name) AS children
    """
    result = tx.run(query, root_node_name=root_node_name)
    record = result.single()
    if record:
        name = record["name"]
        children_names = [child for child in record["children"] if child is not None]
        children = []
        for child_name in children_names:
            child_tree = get_tree(tx, child_name)  # 递归获取子节点
            children.append(child_tree)
        return {"name": name, "children": children}
    else:
        return {"name": root_node_name, "children": []}  # 如果没有找到节点，返回空的子节点列表

In [6]:
import json
def get_and_display_tree(root_node_name):
    with driver.session() as session:
        tree = session.read_transaction(get_tree, root_node_name)
        json_tree = json.dumps(tree, indent=2, ensure_ascii=False)
        return json_tree

In [7]:
requirements = [
'Synchronous and Asyncronous Endpoints',
'CSpace',
'IPC Capability Transfers',
'Invocation Atomicity and Consistency',
'System Call Behavior: Specify the behavior of blocking and non-blocking kernel object invocations, including system call variants and configuration-dependent behavior.'
]

In [8]:
for query in requirements:
    content = retriever.invoke(query)[0].page_content
    with driver.session() as session:
        tree = get_and_display_tree(content)
        with open(f'./requirements/{query}.txt','w') as f:
            f.write(str(tree))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_4129930/2567466667.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_4129930/3584044778.py:4: DeprecationWarning: read_transaction has been renamed to execute_read
  tree = session.read_transaction(get_tree, root_node_name)
/tmp/ipykernel_4129930/2567466667.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_4129930/3584044778.py:4: DeprecationWarning: read_transaction has been renamed to execu